In [32]:
%reload_ext autoreload
%autoreload 2

from IPython.core.display import Markdown
from tqdm.auto import tqdm
import config
import docs
import pandas as pd
import requests

pd.set_option('display.max_colwidth', None)

INDICATOR = 'income_categories'
CONFIG = config.get_config(INDICATOR, '../config.toml')

display(Markdown('## Raw data path'))
raw_dir_path = str(CONFIG['raw_dir']).replace('\\', '/')
display(Markdown(f"[{raw_dir_path}]({raw_dir_path})"))


## Raw data path

[C:/Users/tan/src/regional-pm-2023/data/raw/housing/housing_development/income_categories](C:/Users/tan/src/regional-pm-2023/data/raw/housing/housing_development/income_categories)

# Housing: Housing Costs

## Income Categories

In [33]:
docs.describe_indicator(
    indicator=INDICATOR,
    indicators_xlsx_path=CONFIG['indicators_xlsx_path'],
)

Housing units permitted by income category.

nan

In [34]:
docs.list_schema(
    indicator=INDICATOR,
    indicators_xlsx_path=CONFIG['indicators_xlsx_path'],
)

,name,description,type
column,,,
year,Year,Year of record.,datetime
income_category,Income Category,"Income category of record (""Very Low Income"", ""Low Income"", ""Moderate Income"", ""Above Moderate Income"").",string
new_units_permitted,New Units Permitted,"Housing units permitted, for a given year and income category.",int


In [35]:
docs.list_sources(
    indicator=INDICATOR,
    indicators_xlsx_path=CONFIG['indicators_xlsx_path'],
)

,name,organization,active,notes
source,,,,
hcd_data,HCD Jurisdictions APR Data by Year (ODP),California Department of Housign and Community Development,True,Housing team has a dashboard on SANDAG's internal ODP with data for this.


In [36]:
steps = docs.list_update_steps(
    indicator=INDICATOR,
    indicators_xlsx_path=CONFIG['indicators_xlsx_path'],
)
steps

,step
0,Download income category data from
1,NaN


In [37]:
docs.list_remarks(
    indicator=INDICATOR,
    indicators_xlsx_path=CONFIG['indicators_xlsx_path'],
)

,author,note
0,NaN,NaN


### Step 0: Download data

In [38]:
display(steps.loc[0])

step    Download income category data from
Name: 0, dtype: object

Data was pulled from the Department of Housing and Community Development data used on [housing planner's dashboard](https://internaldata.sandag.org/Other/HCD-Jurisdictions-APR-Data-by-Year/ucti-9k2n).

## Step 1: Aggregate data

In [39]:
display(steps.loc[1])

step    NaN
Name: 1, dtype: object

In [40]:
columns = {
    'YEAR': 'year',
    'Total Very Low Income Units Permitted': 'Very Low Income',
    'Total Low Income Units Permitted': 'Low Income Units',
    'Total Moderate Income Units Permitted': 'Moderate Income Units',
    'Total Above Moderate Income Units Permitted': 'Above Moderate Income Units',
}

In [41]:
income_categories = (
    pd.read_csv(CONFIG['raw_dir']/'HCD_Jurisdictions_APR_Data.csv')
    [list(columns.keys())]
    .rename(columns=columns)
    .melt('year', var_name='income_category', value_name='new_units_permitted')
    .groupby(['year', 'income_category'])
    .sum()
)
income_categories.tail(4)

new_units_permitted
year income_category                                 
2022 Above Moderate Income Units                 8319
     Low Income Units                             957
     Moderate Income Units                       1018
     Very Low Income                              781

### Save Data

In [42]:
display(Markdown('#### Clean data path'))
clean_dir_path = str(CONFIG['clean_dir']).replace('\\', '/')
display(Markdown(f"[{clean_dir_path}]({clean_dir_path})"))

#### Clean data path

[C:/Users/tan/src/regional-pm-2023/data/clean/housing/housing_development/income_categories](C:/Users/tan/src/regional-pm-2023/data/clean/housing/housing_development/income_categories)

In [43]:
income_categories.to_csv(
    CONFIG['clean_dir']
    / f'{INDICATOR}_odp.csv'
)